In [4]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LinearRegression


from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier




from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor




from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    confusion_matrix, classification_report

from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve

from sklearn.model_selection import learning_curve

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.inspection import permutation_importance
#import warnings
#warnings.simplefilter(action="ignore")

# First Baseline

## Regression lineaire avec outliers

In [8]:
dataset1 = pd.read_csv("DATA/df_encod.csv", sep=',')
data1 = dataset1.drop('Unnamed: 0', axis =1)

In [9]:
# define X, y
y1 = dataset1['median_house_value']
x1 = data1.drop('median_house_value', axis =1)

In [11]:
# Create an instance of a linear regression model and fit it to the data with the fit() function:
model1 = LinearRegression().fit(x1, y1) 

# The following section will get results by interpreting the created instance: 

# Obtain the coefficient of determination by calling the model with the score() function, then print the coefficient:
r_sq = model1.score(x1, y1)
print('coefficient of determination:', r_sq)

# Print the Intercept:
print('intercept:', model1.intercept_)

# Print the Slope:
print('slope:', model1.coef_) 

# Predict a Response and print it:
y_pred1 = model1.predict(x1)
print('Predicted response:', y_pred1, sep='\n')

coefficient of determination: 0.6469490011323338
intercept: -2162591.5413745996
slope: [-2.59641026e+04 -2.48449264e+04  1.04884603e+03 -4.95804783e+00
  7.01694945e+01 -3.85490289e+01  7.92461808e+01  3.88202895e+04
 -2.67115858e+04 -6.69765896e+04  1.46474863e+05 -2.99475191e+04
 -2.28391691e+04]
Predicted response:
[ 35430.6490017  317386.949328    60012.92021908 ... 152597.98004801
 178550.81039786 150093.57439964]


# Iteration 2

## Regression lineaire sans outliers

In [12]:
dataset2 = pd.read_csv("DATA/df_encod_sans_outliers.csv", sep=',')
data2 = dataset2.drop('Unnamed: 0', axis =1)

In [14]:
# define X, y
y2 = dataset2['median_house_value']
x2 = data2.drop('median_house_value', axis =1)

In [15]:
# Create an instance of a linear regression model and fit it to the data with the fit() function:
model2 = LinearRegression().fit(x2, y2) 

# The following section will get results by interpreting the created instance: 

# Obtain the coefficient of determination by calling the model with the score() function, then print the coefficient:
r_sq2 = model2.score(x2, y2)
print('coefficient of determination:', r_sq2)

# Print the Intercept:
print('intercept:', model2.intercept_)

# Print the Slope:
print('slope:', model2.coef_) 

# Predict a Response and print it:
y_pred2 = model2.predict(x2)
print('Predicted response:', y_pred2, sep='\n')

coefficient of determination: 0.618593316227131
intercept: -1920466.9825578732
slope: [-2.28541279e+04 -2.10075867e+04  6.91829243e+02 -4.52124019e+00
  5.33700973e+01 -2.76142716e+01  6.25926669e+01  3.47952557e+04
 -1.68186921e+04 -5.73281905e+04  1.13147930e+05 -2.36915525e+04
 -1.53094948e+04]
Predicted response:
[ 48691.86412554 300396.75988997  66669.49002385 ... 151254.00565739
 172019.07684258 150106.19644668]
